# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [66]:
import numpy as np
import pandas as pd
from statsmodels.stats.weightstats import ztest as ztest
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_ind, norm, t, sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [4]:
# Run this code:

salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [5]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [6]:
salaries.isna().sum() 

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [7]:
salaries["Salary or Hourly"].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [8]:
salaries["Department"].value_counts()


POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [9]:
hourly_wage = salaries["Hourly Rate"].dropna()
# H0 = 30
# Ha != 30
statistic, p_value = ztest(hourly_wage, value = 30, alternative='two-sided')
p_value

1.8228873859286195e-94

In [10]:
# Decision
if p_value < 0.05:
    print("Rejection of H0. Keep the alternative: The hourly wage of all hourly workers is significantly different from $30/hr")
if p_value > 0.05:
    print("We don't have enough evidence to reject H0, the salary is not significantly different from $30/hr")

Rejection of H0. Keep the alternative: The hourly wage of all hourly workers is significantly different from $30/hr


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [11]:
annual_salary = salaries[salaries["Department"] == "POLICE"]["Annual Salary"].dropna()
last_year_mean = 86000

# H0 <= 86000
# Ha > 86000

statistic, p_value = ztest(annual_salary, value = last_year_mean, alternative='larger')
p_value

0.0010280845287547855

In [12]:
# Decision
if p_value < 0.025:
    print("Rejection of H0. Keep the alternative: The annual salary mean is indeed larger than last year")
if p_value > 0.025:
    print("We don't have enough evidence to reject H0, the annual salary mean is not larger than last year")

Rejection of H0. Keep the alternative: The annual salary mean is indeed larger than last year


Using the `crosstab` function, find the department that has the most hourly workers. 

In [13]:
pd.crosstab(salaries["Department"], salaries["Salary or Hourly"])["Hourly"].sort_values(ascending = False)

Department
STREETS & SAN            1862
WATER MGMNT              1513
OEMC                     1273
AVIATION                 1082
GENERAL SERVICES          765
TRANSPORTN                725
PUBLIC LIBRARY            299
FAMILY & SUPPORT          287
CITY COUNCIL               64
FINANCE                    44
LAW                        40
ANIMAL CONTRL              19
POLICE                     10
MAYOR'S OFFICE              8
BUSINESS AFFAIRS            7
CULTURAL AFFAIRS            7
COMMUNITY DEVELOPMENT       4
HUMAN RESOURCES             4
HEALTH                      3
PROCUREMENT                 2
FIRE                        2
BUDGET & MGMT               2
HUMAN RELATIONS             0
INSPECTOR GEN               0
DoIT                        0
LICENSE APPL COMM           0
DISABILITIES                0
COPA                        0
CITY CLERK                  0
POLICE BOARD                0
BUILDINGS                   0
BOARD OF ETHICS             0
BOARD OF ELECTION           0

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [14]:
# I will use a z-test because we only have one sample with N > 30.

In [15]:
salaries_department_most_hourly = salaries[salaries["Department"] == "STREETS & SAN"]["Hourly Rate"].dropna()
complain_hourly_salary = 35

# H0 >= 35
# Ha < 35

statistic, p_value = ztest(salaries_department_most_hourly, value = complain_hourly_salary, alternative='smaller')
p_value

5.478855641943434e-22

In [16]:
# Decision
if p_value < 0.025:
    print("Rejection of H0. Keep the alternative: Workers make less than $35/hour")
if p_value > 0.025:
    print("We don't have enough evidence to reject H0, Workers are wrong, they make more than $35/hour")

Rejection of H0. Keep the alternative: Workers make less than $35/hour


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

$$\left[ \hat\mu -1.96 * SE, \hat\mu +1.96 * SE \right]$$

In [51]:
hourly_rate = salaries["Hourly Rate"].dropna()

In [52]:
hourly_rate_mean = hourly_rate.mean()
hourly_rate_mean

32.78855771628128

In [53]:
empirical_standard_error = np.std(hourly_rate) / np.sqrt(hourly_rate.size)
empirical_standard_error

0.13522842712401323

In [54]:
#Calculated using the formula
hourly_rate_mean - 1.96 * empirical_standard_error, hourly_rate_mean + 1.96 * empirical_standard_error

(32.52350999911822, 33.053605433444346)

In [63]:
#Calculated using scipy's t.interval
#t.interval(confidence, df, loc=0, scale=1)
confidence = 0.95
df = hourly_rate.size # df = degrees of freedom, not dataframe...
loc = hourly_rate.mean()
scale = sem(hourly_rate)

hourly_rate_interval = t.interval(confidence, df, loc=loc, scale=scale)
hourly_rate_interval


(32.523458349872726, 33.05365708268984)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [62]:
only_police = salaries["Department"] == "POLICE"
only_salaried = salaries["Salary or Hourly"] == "Salary"

only_police_salaried = salaries[only_police & only_salaried]["Annual Salary"].dropna()

confidence = 0.95
df = only_police_salaried.size # df = degrees of freedom, not dataframe...
loc = only_police_salaried.mean()
scale = sem(only_police_salaried)

only_police_salaried_interval = t.interval(confidence, df, loc=loc, scale=scale)
only_police_salaried_interval

(86177.05631740222, 86795.77268886456)

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [81]:
nobs = salaries["Salary or Hourly"].shape[0]
count = salaries[salaries["Salary or Hourly"] == "Hourly"].shape[0]

In [85]:
H0 = "Number of hourly workers == 25%"
Ha = "Number of hourly workers is significantly different from 25%"

_, p_value = proportions_ztest(count, nobs, value=0.25, alternative='two-sided', prop_var=False)
p_value

0.0004481127249057967

In [86]:
if p_value < 0.025:
    print(f"Rejection of H0. Keep the alternative: {Ha}")
if p_value > 0.025:
    print(f"We don't have enough evidence to reject H0, {H0}")

Rejection of H0. Keep the alternative: Number of hourly workers is significantly different from 25%
